<a href="https://colab.research.google.com/github/claredavies/ArtificialIntelligience/blob/main/ComputationalIntelligienceAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy
import pandas as pd


"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""


'\nThe y=target is to maximize this equation ASAP:\n    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6\n    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)\n    What are the best values for the 6 weights w1 to w6?\n    We are going to use the genetic algorithm for the best possible values after a number of generations.\n'

In [2]:
# genetic algorithm search of the one max optimization problem
from numpy.random import randint
from numpy.random import rand

# Functions

In [3]:
def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

In [4]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

In [5]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [6]:
def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

# Genetic Algorithm

In [7]:
# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = 6

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""

sol_per_pop = 8
num_parents_mating = 4

# Defining the population size.
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
print(new_population)

[[ 2.71026972e+00  9.04858975e-01 -1.09194446e+00  1.65885406e+00
  -2.99523547e+00 -2.68661189e+00]
 [ 3.08396057e+00 -3.28920546e-01 -2.60712501e+00 -3.19345295e+00
   1.72131161e+00 -8.70426677e-01]
 [-2.93450972e+00  8.44599873e-04 -2.88940530e+00  2.81989833e+00
   3.06171263e+00  1.30852681e+00]
 [-1.07654558e+00 -3.74098262e-01 -3.05927329e+00  5.53516057e-01
  -1.47196822e+00 -5.19973529e-01]
 [ 1.64447289e+00 -2.70667542e+00  1.04647272e+00  2.67108362e+00
  -3.11268279e+00 -3.69870409e+00]
 [-6.08705542e-01 -2.25746741e+00 -6.53153370e-01  3.49614989e+00
  -2.33798584e+00  3.77058551e+00]
 [-3.00091988e+00  3.30933839e+00 -3.76235610e+00 -2.78463526e+00
   2.12334444e+00 -3.20755832e+00]
 [-8.93323845e-01  1.74950221e+00 -5.62343619e-01  2.74060635e+00
  -4.60070912e-01  2.60529634e+00]]


In [8]:
def evolutionary_algorithm(num_generations):
  for generation in range(num_generations):
      print("Generation : ", generation)
      # Measing the fitness of each chromosome in the population.
      fitness = cal_pop_fitness(equation_inputs, new_population)

      # Selecting the best parents in the population for mating.
      parents = select_mating_pool(new_population, fitness, 
                                        num_parents_mating)

      # Generating next generation using crossover.
      offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))

      # Adding some variations to the offsrping using mutation.
      offspring_mutation = mutation(offspring_crossover)

      # Creating the new population based on the parents and offspring.
      new_population[0:parents.shape[0], :] = parents
      new_population[parents.shape[0]:, :] = offspring_mutation

      # The best result in the current iteration.
      print("Best result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))

  # Getting the best solution after iterating finishing all generations.
  #At first, the fitness is calculated for each solution in the final generation.
  fitness = cal_pop_fitness(equation_inputs, new_population)
  # Then return the index of that solution corresponding to the best fitness.
  best_match_idx = numpy.where(fitness == numpy.max(fitness))

  print("Finished Iterations")
  print("Best solution : ", new_population[best_match_idx, :])
  print("Best solution fitness : ", fitness[best_match_idx])

  return new_population[best_match_idx, :]

# Inputs

Class Info

In [9]:
#Creating Series
class_series = pd.Series(['Class1', 'Class1', 'Class1', 'Class1','Class2', 'Class2', 'Class2', 'Class2'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'English', 'History', 'Physics'])
hours_series = pd.Series([3, 4, 5, 3, 2, 4, 5, 3])


#Creating Dictionary
frame = { 'Class': class_series, 'Subject': subject_series, 'Hours': hours_series }
#Creating Dataframe
class_subject_hours_dataframe = pd.DataFrame(frame)

#Printing dataframe
print(class_subject_hours_dataframe)

    Class    Subject  Hours
0  Class1      Maths      3
1  Class1    English      4
2  Class1  Chemistry      5
3  Class1    History      3
4  Class2      Maths      2
5  Class2    English      4
6  Class2    History      5
7  Class2    Physics      3


Teacher Info

In [10]:
teacher_series = pd.Series(['ProfessorA', 'ProfessorA', 'ProfessorB', 'ProfessorB','ProfessorC', 'ProfessorC'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'Physics'])

#Creating Dictionary
frame = { 'Teacher': teacher_series, 'Subject': subject_series}
#Creating Dataframe
teacher_subject_dataframe = pd.DataFrame(frame)

teacher_subject_array = teacher_subject_dataframe.to_numpy()
#Printing dataframe
print(teacher_subject_array)

[['ProfessorA' 'Maths']
 ['ProfessorA' 'English']
 ['ProfessorB' 'Chemistry']
 ['ProfessorB' 'History']
 ['ProfessorC' 'Maths']
 ['ProfessorC' 'Physics']]


Define Timetable

In [11]:
startTime = 10
durationLesson = 1
lunchTime = 12
durationLunch = 1
endTime = 15

singleTimetable = []
daysOfWeek = ['Mon','Tue','Wed', 'Thur']

for day in daysOfWeek:
  for time in range(startTime, endTime, durationLesson):
    if time != lunchTime:
      singleTimetable.append([day, time])

print(singleTimetable)

[['Mon', 10], ['Mon', 11], ['Mon', 13], ['Mon', 14], ['Tue', 10], ['Tue', 11], ['Tue', 13], ['Tue', 14], ['Wed', 10], ['Wed', 11], ['Wed', 13], ['Wed', 14], ['Thur', 10], ['Thur', 11], ['Thur', 13], ['Thur', 14]]


In [12]:
def create_timetables(class_subject_hours_dataframe, timetable):
  all_timetables = []
  blank_timetable = [None] * len(timetable)
  number_class_timetables = class_subject_hours_dataframe.Class.nunique()

  for x in range(number_class_timetables):
    all_timetables.append(blank_timetable)

  print(all_timetables)

  return all_timetables

In [43]:
def make_array_1_hour_class(class_mask):
  list_classes_1hr = []
  for index, row in class_mask.iterrows():
    number_hours_per_class = row['Hours']
    for x in range(number_hours_per_class):
      list_classes_1hr.append(row['Subject'])
  list_finished = list_classes_1hr.copy()
  return list_finished

In [49]:
def generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, timetable, populationSize):
  all_timetables = create_timetables(class_subject_hours_dataframe, timetable)
  class_names = class_subject_hours_dataframe.Class.unique()
  number_class_names = class_subject_hours_dataframe.Class.nunique()
  for class_name in class_names:
    mask = class_subject_hours_dataframe.loc[class_subject_hours_dataframe['Class'] == class_name]
    list_classes_1hr = make_array_1_hour_class(mask)
    length = len(list_classes_1hr)
   
    for i in range(length):
        print(list_classes_1hr[i])


  # figure out number of distinct classes
  # use pop
  # need to randomly assign based of professor number


In [50]:
generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, 4)

[[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]]
Maths
Maths
Maths
English
English
English
English
Chemistry
Chemistry
Chemistry
Chemistry
Chemistry
History
History
History
Maths
Maths
English
English
English
English
History
History
History
History
History
Physics
Physics
Physics


# Execution

In [38]:
num_generations = 5

evolutionary_algorithm(num_generations)

Generation :  0
Best result :  134.30444802581837
Generation :  1
Best result :  134.30444802581837
Generation :  2
Best result :  138.95503532987385
Generation :  3
Best result :  142.72556515422528
Generation :  4
Best result :  150.93878864210043
Finished Iterations
Best solution :  [[[ -1.62969513  -2.97072974   2.30087151   2.18980049 -10.44629408
    -3.74570684]]]
Best solution fitness :  [150.93878864]


array([[[ -1.62969513,  -2.97072974,   2.30087151,   2.18980049,
         -10.44629408,  -3.74570684]]])